In [1]:
import cobra
import pandas as pd

ecoli_data = pd.read_csv("data/ecoli_data.csv")
ecoli_dict = dict(zip(ecoli_data["# Reaction ID"], ecoli_data[" reaction activity [mmol/gDW/h] "]))


model = cobra.io.load_json_model("data/ecoli_data_js.json")
model

Name,e_coli_core
Memory address,13977f500
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


## Exercise 1

In [2]:
resultOne = ecoli_data.loc[ecoli_data["# Reaction ID"] == "G6PDH2r"]
resultOne

,# Reaction ID,reaction activity [mmol/gDW/h]
50,G6PDH2r,5.68


In [3]:
resultTwo = ecoli_data.loc[ecoli_data["# Reaction ID"] == "PGL"]
resultTwo

,# Reaction ID,reaction activity [mmol/gDW/h]
4,PGL,8.12


a) As each reaction has a flux no longer under mass balance, but instead equal to its individual maximum capacity, the reaction fluxes in a linear pathway are no longer the same (the output of one reaction was equal to the input in the next one). For example the reactions: G6PDH2r -> PGL, each carrying a flux of 4.96 mmol/gDW/h under mass balance, now have the following flux values G6PDH2r: 5.68 mmol/gDW/h and PGL: 8.12 mmol/gDW/h. This is because the maximal activity levels are constrained by the enzyme abundance and not by the stochiometric mass balance, that was limiting the flux.

b) Below the reactions with no gene expression data are shown (taken from the non-interactive E. coli core ESCHER model):
1. EX_glc__D_e - it is an exchange reaction representing the intake of glucose. Gene expression data is not applicable to exchange reactions, as they don't have any genes associated with them. They only represent the boundary conditions of the system (how much metabolite is available at the boundary), so there is no gene to map to (GPR rules).
2. FORti - it is a transport reaction. It is transporing Formate from extracellular space to the cytosol (for_e <=> for_c). It doesn't have any gene encoding an enzyme that directly catalyses it and so gene expression data is irrelevant here. This reaction only depends on the physical properties of the cell.
3. BIOMASS_Ecoli_core_w_GAM - it is a biomass reaction representing the objective function. They work by combining certain metabolites from the entire network to simulate cell growth. Gene expression data is not applicable to this type of reactions, as they are not catalyzed by a single enzyme and so they cannot be mapped to a single gene (GPR rule).

## Exercise 2

In [4]:
all_reactions = model.reactions

for reaction in all_reactions:
    if reaction.id not in ecoli_dict:# if there is no data for the reaction, don't do anything (leave their default constrains)
        continue  
    value = ecoli_dict[reaction.id]
    if reaction.id == "EX_glc__D_e": # if its the glucose exchange reaction, set the upper_bound to 1000 (high absolute default)
        reaction.upper_bound = 1000
        continue  
    if reaction.id.lower() == "atpm": # if its the ATP maintenance reaction, don't change anything
        continue  
    if reaction.reversibility: # if the reaction is reversble, set the lower and upper bound to +- max values
        reaction.lower_bound = -value
        reaction.upper_bound = value
    else: # if the reaction is irreversible, set the lower bound to 0 and the upper to max value
        reaction.lower_bound = 0
        reaction.upper_bound = value

In [5]:
print("ID", "Lower_bound", "Upper_bound")
for reaction in all_reactions:
    print(reaction.id, reaction.lower_bound, reaction.upper_bound)

ID Lower_bound Upper_bound
PFK 0 12.12
PFL 0 1.0
PGI -13.12 13.12
PGK -23.13 23.13
PGL 0 8.12
ACALD -1.16 1.16
AKGt2r -3.1 3.1
PGM -20.01 20.01
PIt2r -6.03 6.03
ALCD2x -9.01 9.01
ACALDt -2.29 2.29
ACKr -1.19 1.19
PPC 0 2.56
ACONTa -25.35 25.35
ACONTb -25.35 25.35
ATPM 8.39 1000.0
PPCK 0 25.23
ACt2r -3.23 3.23
PPS 0 2.5
ADK1 -30.57 30.57
AKGDH 0 24.35
ATPS4r -60.5 60.5
PTAr -4.47 4.47
PYK 0 26.78
BIOMASS_Ecoli_core_w_GAM 0.0 1000.0
PYRt2 -1.26 1.26
CO2t -30.45 30.45
RPE -5.67 5.67
CS 0 20.56
RPI -5.56 5.56
SUCCt2_2 0 2.36
CYTBD 0 40.56
D_LACt2 -4.56 4.56
ENO -28.78 28.78
SUCCt3 0 6.67
ETOHt2r -2.34 2.34
SUCDi 0 24.34
SUCOAS -20.6 20.6
TALA -4.45 4.45
THD2 0 4.5
TKT1 -3.34 3.34
TKT2 -3.34 3.34
TPI -45.56 45.56
EX_ac_e 0.0 1000.0
EX_acald_e 0.0 1000.0
EX_akg_e 0.0 1000.0
EX_co2_e -1000.0 1000.0
EX_etoh_e 0.0 1000.0
EX_for_e 0.0 1000.0
EX_fru_e 0.0 1000.0
EX_fum_e 0.0 1000.0
EX_glc__D_e -10.0 1000.0
EX_gln__L_e 0.0 1000.0
EX_glu__L_e 0.0 1000.0
EX_h_e -1000.0 1000.0
EX_h2o_e -1000.0 1000.0

## Exercise 3

In [6]:
# a) FVA for all reactions

FVA_results = cobra.flux_analysis.variability.flux_variability_analysis(model = model, reaction_list = model.reactions, loopless = False, fraction_of_optimum = 0.0)

for rxn_id, row in FVA_results.iterrows():
    print(f"{rxn_id}: min = {row['minimum']:.4f}, max = {row['maximum']:.4f}")

PFK: min = 0.3347, max = 10.9536
PFL: min = 0.0000, max = 0.0000
PGI: min = -0.0853, max = 8.8200
PGK: min = -17.1960, max = -0.8794
PGL: min = 0.0000, max = 4.4961
ACALD: min = -1.1600, max = 0.0000
AKGt2r: min = -3.1000, max = 0.0000
PGM: min = -16.6389, max = -0.8794
PIt2r: min = 0.0000, max = 1.7642
ALCD2x: min = -1.1600, max = 0.0000
ACALDt: min = -1.1600, max = 0.0000
ACKr: min = -1.1900, max = -0.0000
PPC: min = 0.0000, max = 2.5600
ACONTa: min = 0.0000, max = 5.5533
ACONTb: min = 0.0000, max = 5.5533
ATPM: min = 8.3900, max = 39.8275
PPCK: min = 0.0000, max = 4.6900
ACt2r: min = -1.1900, max = 0.0000
PPS: min = 0.0000, max = 2.5000
ADK1: min = 0.0000, max = 2.5000
AKGDH: min = 0.0000, max = 3.9345
ATPS4r: min = 0.9120, max = 27.6410
PTAr: min = 0.0000, max = 1.1900
PYK: min = 0.0000, max = 9.1320
BIOMASS_Ecoli_core_w_GAM: min = 0.0000, max = 0.4796
PYRt2: min = -1.2600, max = 0.0000
CO2t: min = -13.6748, max = 1.0153
RPE: min = -0.3421, max = 2.6732
CS: min = 0.0000, max = 5.55

In [7]:
# b) 

restricted_reactions = []

for rxn in model.reactions:
    
    fva_max = FVA_results.loc[rxn.id, "maximum"]
    if fva_max != 0 and fva_max < rxn.upper_bound:
        restricted_reactions.append(rxn.id)

print(f"Reactions with gene expression-imposed maximal reaction activities: {len(restricted_reactions)}")
print(restricted_reactions)

Reactions with gene expression-imposed maximal reaction activities: 57
['PFK', 'PGI', 'PGK', 'PGL', 'PGM', 'PIt2r', 'ACKr', 'ACONTa', 'ACONTb', 'ATPM', 'PPCK', 'ADK1', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK', 'BIOMASS_Ecoli_core_w_GAM', 'CO2t', 'RPE', 'CS', 'CYTBD', 'ENO', 'SUCDi', 'TALA', 'TKT1', 'TKT2', 'TPI', 'EX_ac_e', 'EX_acald_e', 'EX_akg_e', 'EX_co2_e', 'EX_etoh_e', 'EX_glc__D_e', 'EX_glu__L_e', 'EX_h_e', 'EX_h2o_e', 'EX_lac__D_e', 'EX_nh4_e', 'EX_o2_e', 'EX_pi_e', 'EX_pyr_e', 'EX_succ_e', 'FBA', 'FUM', 'G6PDH2r', 'GAPD', 'GLCpts', 'GLUDy', 'GLUN', 'GLUSy', 'GND', 'H2Ot', 'ICDHyr', 'ICL', 'MALS', 'O2t', 'PDH']


57 reactions behave this way. They do so because of the network’s structure and mass balance constraints. The flux states of other reactions act as bottlenecks, which prevent the reaction from achieving its theoretical maximum flux, even though its upper bound would technically allow it. That is to say, the local enzyme capacity does not guarantee a high flux through a reaction. The actual flux is determined by the global network topology and the overall metabolic state of the cell.

In [8]:
# c) 

positive_min_reactions = FVA_results[FVA_results["minimum"] > 0]

print(f"Reactions with positive minimal flux: {len(positive_min_reactions)}")
print(positive_min_reactions)

Reactions with positive minimal flux: 16
               minimum    maximum
PFK       3.346774e-01  10.953556
ATPM      8.390000e+00  39.827500
ATPS4r    9.120000e-01  27.641000
CS        5.151944e-16   5.553333
CYTBD     1.026667e+00  24.312000
ENO       8.793548e-01  16.638932
TPI       3.346774e-01   8.603556
EX_h_e    8.108831e-15  20.402166
EX_h2o_e  1.082222e+00  18.590873
FBA       3.346774e-01   8.603556
GAPD      8.793548e-01  17.196002
GLCpts    4.794286e-01   9.293300
ICDHyr    2.790937e-15   5.012277
NADH16    2.164444e+00  20.260000
O2t       5.133333e-01  12.156000
PDH       2.546667e-01  10.931107


15 reactions have a positive minimal flux in the FVA. That means that these reactions always need to operate in the forward direction, and therefore have positive flux. They are essential for the metabolic network to function under the specified conditions. Those reactions are part of obligatory pathways that produce a specific biomass objective to satisfy other vital demands within the model. We can see for example ATPM (ATP maintenance), which always needs to run to satisfy the cell’s energy requirements. 

## Exercise 4

In [12]:
# a) 
solution = model.optimize() 
solution

,fluxes,reduced_costs
PFK,7.314456,1.734723e-18
PFL,0.000000,-8.673617e-19
PGI,7.376438,0.000000e+00
PGK,-14.466494,1.912182e-18
PGL,0.475103,1.301043e-18
...,...,...
NADH16,20.260000,5.131667e-02
NADTRHD,0.000000,-1.710556e-02
NH4t,2.615007,0.000000e+00
O2t,11.516286,0.000000e+00


In [13]:
max_biomass_production_rate = solution.objective_value
print("Maximal biomass production rate in the presence of the implemented constraints:", max_biomass_production_rate)

Maximal biomass production rate in the presence of the implemented constraints: 0.4795714305019132


In [17]:
# b)
fluxes = solution.fluxes # Get flux distribution
bottlenecks = []
tolerance = 1e-6  # Numerical tolerance to account for floating point inaccuracies

for rxn in model.reactions:
    flux = fluxes[rxn.id]
    if rxn.upper_bound < float("inf"):  # Make sure the reaction is not unconstrained, because it couldn't be a bottleneck in that case
        if abs(flux - rxn.upper_bound) < tolerance:
            bottlenecks.append((rxn.id, flux, rxn.upper_bound))

print("Bottleneck reactions:")
for rxn_id, flux, ub in bottlenecks:
    print(f"Reaction = {rxn_id}, flux = {flux}, upper_bound = {ub}")

Bottleneck reactions:
Reaction = THD2, flux = 4.5, upper_bound = 4.5
Reaction = NADH16, flux = 20.26, upper_bound = 20.26


In [23]:
# c)
constrained_reactions = [rxn for rxn in model.reactions if rxn.upper_bound < float('inf')]
zero_flux_constrained_reactions = [rxn for rxn in constrained_reactions if abs(fluxes[rxn.id]) < tolerance]

print("Inactive constrained reactions:")
for rxn in zero_flux_constrained_reactions:
    print(rxn.id)

Inactive constrained reactions:
PFL
AKGt2r
ACALDt
PPCK
PPS
ADK1
SUCCt2_2
SUCCt3
EX_acald_e
EX_akg_e
EX_for_e
EX_fru_e
EX_fum_e
EX_gln__L_e
EX_glu__L_e
EX_mal__L_e
EX_succ_e
FBP
FORt2
FORt
FRD7
FRUpts2
FUMt2_2
GLNabc
GLUN
GLUSy
GLUt2r
ICL
MALS
MALt2_2
ME1
ME2
NADTRHD
